In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

# Load the user-song interaction data (e.g., using pandas)
data = pd.read_csv("/content/Training_set_songs.csv")

# Create a user-item matrix
user_item_matrix = data.pivot(index='user_id', columns='song_id', values='rating')

# Fill missing values with 0 (assuming unplayed songs have a rating of 0)
user_item_matrix = user_item_matrix.fillna(0)

# Calculate user-user similarity using cosine similarity
user_similarity = 1 - pairwise_distances(user_item_matrix, metric='cosine')

# Define a function to generate personalized playlists
def generate_playlist(user_id, num_songs=10):
    # Find the top N most similar users to the given user
    user_similarities = user_similarity[user_id]
    similar_users = user_similarities.argsort()[::-1][1:]

    # Find songs that the user has not listened to but the similar users liked
    user_songs = user_item_matrix.loc[user_id]
    playlist = []
    for song_id in user_item_matrix.columns:
        if user_songs[song_id] == 0:
            song_score = sum(user_item_matrix.loc[similar_user][song_id] * user_similarities[similar_user] for similar_user in similar_users)
            playlist.append((song_id, song_score))

    # Sort the playlist by song score and select the top N songs
    playlist.sort(key=lambda x: x[1], reverse=True)
    recommended_songs = [song_id for song_id, _ in playlist[:num_songs]]

    return recommended_songs

# Example usage
user_id = 1  # Replace with the user ID for whom you want to generate a playlist
playlist = generate_playlist(user_id, num_songs=10)
print(f"Recommended songs for user {user_id}: {playlist}")

KeyError: ignored